<a href="https://githubtocolab.com/juanDev-47/IA-project/blob/main/02%20-%20Preprocesado%20de%20datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
#bibliotecas de machine learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc

In [2]:
# install kaggle
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"juanpabloarenasvlez","key":"5a3c067016d90c54fa052baee280fa72"}'}

In [4]:
# create a kaggle folder
! mkdir ~/.kaggle

In [5]:
# copy the kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

In [6]:
# permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
# to list all datasets in kaggle
! kaggle datasets list

ref                                                    title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
deepcontractor/mcdonalds-india-menu-nutrition-facts    McDonald's India : Menu Nutrition Dataset          5KB  2022-07-28 17:08:52           2419        118  0.9411765        
iamsouravbanerjee/house-rent-prediction-dataset        House Rent Prediction Dataset                     82KB  2022-08-20 13:49:03           2452        101  1.0              
arianazmoudeh/airbnbopendata                           Airbnb Open Data                                  10MB  2022-08-01 15:58:10           1678         59  0.9705882        
ericsims/world-cheese-awards-worlds-cheesiest-dataset  World Cheese Awards (World's Cheesiest Dataset)   49KB  2022-08-1

In [8]:
!kaggle datasets download -d programmerrdai/cancer

 64% 5.00M/7.84M [00:00<00:00, 40.3MB/s]
100% 7.84M/7.84M [00:00<00:00, 58.4MB/s]


In [9]:
!unzip cancer.zip

Archive:  cancer.zip
  inflating: annual-number-of-deaths-by-cause.csv  
  inflating: breast-cancer-survival-rates-vs-gdp-per-capita.csv  
  inflating: cancer-death-rates-by-age.csv  
  inflating: cancer-death-rates-by-type.csv  
  inflating: cancer-death-rates-in-the-us.csv  
  inflating: cancer-death-rates.csv  
  inflating: cancer-deaths-by-age.csv  
  inflating: cancer-deaths-by-type-grouped.csv  
  inflating: cancer-deaths-rate-and-age-standardized-rate-index.csv  
  inflating: death-rate-from-cancers-vs-average-income.csv  
  inflating: disease-burden-rates-by-cancer-types.csv  
  inflating: disease-burden-rates-from-cancers.csv  
  inflating: five-year-survival-rate-from-liver-cancer.csv  
  inflating: five-year-survival-rates-by-cancer-type.csv  
  inflating: five-year-survival-rates-from-breast-cancer.csv  
  inflating: five-year-survival-rates-from-lung-cancer.csv  
  inflating: lung-cancer-deaths-per-100000-by-sex-1950-2002.csv  
  inflating: lung-cancer-survival-rate-vs-gdp

## 1. Lectura y carga del dataset

In [22]:
# upload data
cancer_data = pd.read_csv("annual-number-of-deaths-by-cause.csv")

In [11]:
# view the dataset
cancer_data

,Entity,Code,Year,Number of executions (Amnesty International),Deaths - Meningitis - Sex: Both - Age: All Ages (Number),Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number),Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number),Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number),Deaths - Malaria - Sex: Both - Age: All Ages (Number),Deaths - Drowning - Sex: Both - Age: All Ages (Number),...,Deaths - Chronic kidney disease - Sex: Both - Age: All Ages (Number),Deaths - Poisonings - Sex: Both - Age: All Ages (Number),Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number),Terrorism (deaths),Deaths - Road injuries - Sex: Both - Age: All Ages (Number),Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number),Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number),Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number),"Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)",Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,2007,15,2933.0,1402.0,450.0,2488.0,393.0,2127.0,...,4490.0,513.0,2439.0,1199.0,7425.0,7222.0,3346.0,6458.0,481.0,3437.0
1,Afghanistan,AFG,2008,17,2731.0,1424.0,455.0,2277.0,255.0,1973.0,...,4534.0,495.0,2231.0,1092.0,7355.0,7143.0,3316.0,6408.0,462.0,3005.0
2,Afghanistan,AFG,2009,0,2460.0,1449.0,460.0,2040.0,239.0,1852.0,...,4597.0,483.0,1998.0,1065.0,7290.0,7045.0,3291.0,6358.0,448.0,2663.0
3,Afghanistan,AFG,2011,2,2327.0,1508.0,473.0,1846.0,390.0,1775.0,...,4785.0,483.0,1805.0,1525.0,7432.0,6916.0,3318.0,6370.0,448.0,2365.0
4,Afghanistan,AFG,2012,14,2254.0,1544.0,482.0,1705.0,94.0,1716.0,...,4846.0,482.0,1667.0,3521.0,7494.0,6878.0,3353.0,6398.0,445.0,2264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7268,Zimbabwe,ZWE,2015,NaN,1439.0,754.0,215.0,3019.0,2518.0,770.0,...,2108.0,381.0,2990.0,NaN,2373.0,2751.0,1956.0,4202.0,632.0,146.0
7269,Zimbabwe,ZWE,2016,NaN,1457.0,767.0,219.0,3056.0,2050.0,801.0,...,2160.0,393.0,3027.0,NaN,2436.0,2788.0,1962.0,4264.0,648.0,146.0
7270,Zimbabwe,ZWE,2017,NaN,1460.0,781.0,223.0,2990.0,2116.0,818.0,...,2196.0,398.0,2962.0,0.0,2473.0,2818.0,2007.0,4342.0,654.0,144.0
7271,Zimbabwe,ZWE,2018,NaN,1450.0,795.0,227.0,2918.0,2088.0,825.0,...,2240.0,400.0,2890.0,NaN,2509.0,2849.0,2030.0,4377.0,657.0,139.0


## 4 Limpieza de datos

### 4.1. Remoción de las lecturas cero de la variable objetivo
Como se se mencionó anteriormente al analizar la variable objetivo, se observó que existes muchas lecturas de valor cero para la variable objetivo (Deaths - Meningitis - Sex: Both - Age: All Ages (Number)). Debido a esto se opta por eliminar las filas que contengan un valor cero para la variable objetivo ya que esto indica que no se tiene una disponibilidad de la medición y estos no podrían utilizarse para entrenar.

In [24]:
death_cancer_zero = list(cancer_data[cancer_data['Deaths - Meningitis - Sex: Both - Age: All Ages (Number)'] == 0].index) #lista con los indices de los valores que tienen lecturas 0
cancer_data.drop(death_cancer_zero, axis = 0, inplace = True) #eliminar filas con lecturas cero en la variable objetivo
print('Nuevo tamaño de los datos: ',cancer_data.shape)

Nuevo tamaño de los datos:  (6957, 34)


### 4.2. Eliminación de las columnas con muchos datos faltantes

Del análisis exploratorio de las variables, se encontró que existen variables que contienen gran cantidad de datos faltantes. En este caso se optará por eliminar las columnas en las que los datos faltantes representen el 50% o más de la totalidad de los datos.

In [25]:
criterio = len(cancer_data) * 0.5 #criterio para eliminar la columna (50% de las filas que se tienen)
cancer_data.dropna(axis=1, thresh = criterio, inplace = True) #eliminación de las columnas con 50% o más de datos faltantes
print('New Shape of cancer_data Data:',cancer_data.shape)

New Shape of cancer_data Data: (6957, 34)


### 4.3 relleno de datos faltantes

Anteriormente se optó por eliminar las columnas que poseian el 50% o más en datos en datos faltantes, sin embargo, aun quedan variables con datos faltantes en el dataset. Para estas columnas se rellenaran los valores faltantes al reemplazarlos por la media de los valores con los que se cuenta para cada columna, primero conoceremos quienes son:

In [26]:
cancer_data.isnull().sum()

Entity                                                                                         0
Code                                                                                        1067
Year                                                                                           0
Deaths - Meningitis - Sex: Both - Age: All Ages (Number)                                     433
Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)        433
Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number)                            433
Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)                       433
Deaths - Malaria - Sex: Both - Age: All Ages (Number)                                        433
Deaths - Drowning - Sex: Both - Age: All Ages (Number)                                       433
Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number)                         433
Deaths - Maternal disorders - 

In [36]:
cancer_data['Deaths - Meningitis - Sex: Both - Age: All Ages (Number)'].fillna(cancer_data['Deaths - Meningitis - Sex: Both - Age: All Ages (Number)'].median(), inplace=True)
cancer_data["Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Malaria - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Malaria - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Drowning - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Drowning - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Maternal disorders - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Maternal disorders - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Tuberculosis - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Tuberculosis - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Lower respiratory infections - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Lower respiratory infections - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Neonatal disorders - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Neonatal disorders - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Alcohol use disorders - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Alcohol use disorders - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Self-harm - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Self-harm - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Exposure to forces of nature - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Exposure to forces of nature - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Diarrheal diseases - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Diarrheal diseases - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Environmental heat and cold exposure - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Environmental heat and cold exposure - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Neoplasms - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Neoplasms - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Conflict and terrorism - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Conflict and terrorism - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Diabetes mellitus - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Diabetes mellitus - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Chronic kidney disease - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Chronic kidney disease - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Poisonings - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Poisonings - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Road injuries - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Road injuries - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)
cancer_data["Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number)"].fillna(cancer_data["Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number)"].median(), inplace=True)

In [37]:
cancer_data.isnull().sum()

Entity                                                                                         0
Code                                                                                        1067
Year                                                                                           0
Deaths - Meningitis - Sex: Both - Age: All Ages (Number)                                       0
Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)          0
Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number)                              0
Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)                         0
Deaths - Malaria - Sex: Both - Age: All Ages (Number)                                          0
Deaths - Drowning - Sex: Both - Age: All Ages (Number)                                         0
Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number)                           0
Deaths - Maternal disorders - 

### 4.4. Transformación de las variables categoricas


Las variables categoricas, como lo son en este caso "Entity" pueden ser utiles a la hora de realizar el análisis, sin embargo, no pueden ser usadas en la forma categorica, por lo que se deben convertir en variables numericas que si podamos utilzar para entrenar un modelo.

In [38]:
var_categoricas = ['Entity']
encoder = preprocessing.LabelEncoder()

for i in var_categoricas:
    
    cancer_data[i] = encoder.fit_transform(cancer_data[i])
    
print (cancer_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6957 entries, 0 to 7272
Data columns (total 34 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   Entity                                                                                    6957 non-null   int64  
 1   Code                                                                                      5890 non-null   object 
 2   Year                                                                                      6957 non-null   int64  
 3   Deaths - Meningitis - Sex: Both - Age: All Ages (Number)                                  6957 non-null   float64
 4   Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)     6957 non-null   float64
 5   Deaths - Parkinson's disease - Sex: Both - Age: All Age